#### Artificial Intelligence for Robotics
# Lesson 1: Localization

## Intro

In this course you will learn how to program self-driving cars! Specifically, in this unit, you will
learn how to program a localizer, which is a directional aid used to assist machines in making
informed navigation decisions.
Over the past decade computer scientists have developed methods using sensors, radars and
software to program vehicles with the ability to sense their own location, the locations of
other vehicles and navigate a charted course.
Stanford University Professor and director of the Stanford Artificial Intelligence Lab,
Sebastian Thrun's autonomous cars, Stanley and Junior illustrate the progress that has been
made in this field over the last decade. Additionally, the Google Driverless Car Project seeks to
further research and development to make driverless cars a viable option for people
everywhere. Driverless cars sound neat, huh? Want to program your own?

Let's get started!


## The Problem of Localization
Localization is the ability for a machine to locate itself in space. Consider a robot lost in space. Within its environment, how can the robot locate where it is? Rather than install a GPS device in our robot, we are going to write a program to implement localization. Our localization program will reduce the margin of error considerably compared to a GPS device, whose margin of error can be as high as ten meters. For our robot to successfully and accurately navigate itself through space, we are looking for a margin of error between two and ten centimeters.

Imagine a robot resides in a one-dimensional world, so somewhere along a straight line, with no idea where it is in this world. For an example of such a world, we can imagine a long, narrow hallway where it is only possible to move forward or backwards; sideways motion is impossible. Since our robot is completely clueless about its location, it believes that every point in this one dimensional world is equally likely to be its current position. We can describe this mathematically by saying that the robot's probability function is uniform (the same) over the sample space (in this case, the robot's one-dimensional world).
If we were to draw a graph of this probability function with probability on the vertical axis and location on the horizontal axis, we would draw a straight, level line. This line describes a uniform probability function, and it represents the state of maximum confusion.



<img src = "./Img1.png"/>

Assume there are three landmarks, which are three doors that all look alike and we can distinguish a door from a non-door area.If the robot senses it is next to a door, how does this affect our belief — or the probability that the robot is near a door?

<img src = "./Img2.png">

In the new function, there are three bumps aligned with the location of the doors. Since the robot has just sensed that it is near a door, it assigns these locations greater probability (indicated by the bumps in the graph) whereas, all of the other locations have decreased belief.

This function represents another probability distribution, called the posterior belief where the function is defined after the robot's sense measurement has been taken. Copyright © 2014 Udacity, Inc. All Rights Reserved. The posterior function is the best representation of the robot's current belief, where each bump represents the robot's evaluation of its position relative to a door. However, the possibility of making a bad measurement constantly looms over robotics, and over the course of this class we will see various ways to handle this problem.


In [8]:
p=[0, 1, 0, 0, 0]
world=['green', 'red', 'red', 'green', 'green']
measurements = ['red', 'green']
pHit = 0.6
pMiss = 0.2
pExact = 0.8
pOvershoot = 0.1
pUndershoot = 0.1

def sense(p, Z):
    q=[]
    for i in range(len(p)):
        hit = (Z == world[i])
        q.append(p[i] * (hit * pHit + (1-hit) * pMiss))
    s = sum(q)
    for i in range(len(q)):
        q[i] = q[i] / s
    return q

def move(p, U):
    #
    n = len(p)
    #q=range(len(p))
    q = []
    for i in range(n):
        s = p[(i-U)%len(p)]*pExact 
        s += p[(i-U+1)%len(p)]*pUndershoot 
        s += p[(i-U-1)%len(p)]*pOvershoot        
        q.append(s)
    return q

print move(p, 0)

[0.1, 0.8, 0.1, 0.0, 0.0]


### Localization 2D

 The function localize takes the following arguments:

 #### Colors:
 
        2D list, each entry either 'R' (for red cell) or 'G' (for green cell)

 #### measurements:
 
        list of measurements taken by the robot, each entry either 'R' or 'G'

 #### motions:
 
        list of actions taken by the robot, each entry of the form [dy,dx],
        where dx refers to the change in the x-direction (positive meaning
        movement to the right) and dy refers to the change in the y-direction
        (positive meaning movement downward)
        NOTE: the *first* coordinate is change in y; the *second* coordinate is
              change in x

 #### sensor_right:
 
        float between 0 and 1, giving the probability that any given
        measurement is correct; the probability that the measurement is
        incorrect is 1-sensor_right

 #### p_move:
 
        float between 0 and 1, giving the probability that any given movement
        command takes place; the probability that the movement command fails
        (and the robot remains still) is 1-p_move; the robot will NOT overshoot
        its destination in this exercise

 The function should RETURN (not just show or print) a 2D list (of the same
 dimensions as colors) that gives the probabilities that the robot occupies
 each cell in the world.

 Compute the probabilities by assuming the robot initially has a uniform
 probability of being in any cell.

 Also assume that at each step, the robot:
 1. first makes a movement,
 2. then takes a measurement.

 Motion:
  * [0,0] - stay
  * [0,1] - right
  * [0,-1] - left
  * [1,0] - down
  * [-1,0] - up

In [12]:
def localize(colors,measurements,motions,sensor_right,p_move):
    # initializes p to a uniform distribution over a grid of the same dimensions as colors
    pinit = 1.0 / float(len(colors)) / float(len(colors[0]))
    p = [[pinit for row in range(len(colors[0]))] for col in range(len(colors))]
    
#    p = [[1,0,0,0,0],
#         [0,0,0,0,0],
#         [0,0,0,0,0],
#         [0,0,0,0,0]]
    
    sensor_wrong = 1- sensor_right
    p_stay = 1- p_move
    
    m = len(colors) # no of rows
    n = len(colors[0]) # no of columns
    
    
    # >>> Insert your code here <<<
    def move(p,U):
        q = [[0 for col in range(n)] for row in range(m)]               
        for i in range(m):
            for j in range(n):                                   
                q[i][j] = p[(i-U[0]%m)][(j-U[1]%n)] * p_move + p[i][j]*p_stay
                
        return q

    def sense(p,Z):
        q = [[0 for col in range(n)] for row in range(m)]
        s = 0
        for i in range(m):
            for j in range(n):
                hit = (Z == colors[i][j])
                q[i][j] = p[i][j]*(hit*sensor_right + (1-hit)*sensor_wrong)
                s += q[i][j]    
        
        for i in range(m):
            for j in range(n):
                q[i][j] = q[i][j]/s
        
        return q
        
    for l in range(len(measurements)):
        p = move(p,motions[l])        
        p = sense(p,measurements[l])         
        
    return p

def show(p):
    rows = ['[' + ','.join(map(lambda x: '{0:.5f}'.format(x),r)) + ']' for r in p]
    print '[' + ',\n '.join(rows) + ']'
    
#############################################################
# For the following test case, your output should be 
# [[0.01105, 0.02464, 0.06799, 0.04472, 0.02465],
#  [0.00715, 0.01017, 0.08696, 0.07988, 0.00935],
#  [0.00739, 0.00894, 0.11272, 0.35350, 0.04065],
#  [0.00910, 0.00715, 0.01434, 0.04313, 0.03642]]
# (within a tolerance of +/- 0.001 for each entry)

colors = [['R','G','G','R','R'],
          ['R','R','G','R','R'],
          ['R','R','G','G','R'],
          ['R','R','R','R','R']]
measurements = ['G','G','G','G','G']
motions = [[0,0],[0,1],[1,0],[1,0],[0,1]]
p = localize(colors,measurements,motions,sensor_right = 0.7, p_move = 0.8)
show(p) # displays your answer

[[0.01106,0.02464,0.06800,0.04472,0.02465],
 [0.00715,0.01017,0.08697,0.07988,0.00935],
 [0.00740,0.00894,0.11273,0.35351,0.04066],
 [0.00911,0.00715,0.01435,0.04313,0.03643]]


In [11]:
0.04*20


0.8